In [1]:
import os
os.cpu_count()

2

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("gittoken")

!git clone https://{secret_value}@github.com/moienr/TemporalGAN.git


Cloning into 'TemporalGAN'...
remote: Enumerating objects: 992, done.
remote: Counting objects: 100% (295/295), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 992 (delta 196), reused 257 (delta 158), pack-reused 697
Receiving objects: 100% (992/992), 82.96 MiB | 35.39 MiB/s, done.
Resolving deltas: 100% (662/662), done.


In [3]:
import torch
import numpy as np
from torch.utils.data import Dataset
from glob import glob
from skimage import io
import os
from torchvision import datasets, transforms

In [4]:
import time
sleep_time = 5
while not os.path.exists("/kaggle/working/TemporalGAN"):
    print("didn't find the path, wating {sleep_time} more seconds...")
    time.sleep(sleep_time)
print("path found...")

path found...


In [5]:
torch.__version__

'1.13.0'

In [6]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [7]:
import sys
sys.path.append("/kaggle/working/TemporalGAN")

In [8]:

from dataset.data_loaders import *
from dataset.utils.plot_utils import plot_s1s2_tensors

In [9]:
# transform = transforms.Compose([S2S1Normalize(),myToTensor()])

# print("Reading only S1 2021 train data...")
# s1s2_dataset = Sen12Dataset(s1_t1_dir="E:\\s1s2\\s1s2_patched_light\\s1s2_patched_light\\2021\\s1_imgs\\train",
#                             s2_t1_dir="E:\\s1s2\\s1s2_patched_light\\s1s2_patched_light\\2021\\s2_imgs\\train",
#                             s1_t2_dir="E:\\s1s2\\s1s2_patched_light\\s1s2_patched_light\\2019\\s1_imgs\\train",
#                             s2_t2_dir="E:\\s1s2\\s1s2_patched_light\\s1s2_patched_light\\2019\\s2_imgs\\train",
#                             transform=transform,
#                             two_way=False)
# print("len(s1s2_dataset): ",len(s1s2_dataset))
# print("s1s2_dataset[0][0]shape: ",s1s2_dataset[0][1].shape)

In [10]:
# plot_s1s2_tensors(s1s2_dataset[1], ["s2t2", "s1t2", "s2t1", "s1t1", "change map", "reversed change map"], 3,2)

In [11]:
S2_INCHANNELS = 6
S1_INCHANNELS = 1
LEARNING_RATE = 2e-4
BATCH_SIZE = 1
NUM_WORKERS = 2
IMAGE_SIZE = 256
WEIGHTED_LOSS = True
INPUT_CHANGE_MAP = False
L1_LAMBDA = 100
LAMBDA_GP = 10
NUM_EPOCHS = 5
LOAD_MODEL = False
SAVE_MODEL = False
SAVE_EVERY_EPOCH = 1
CHECKPOINT_DISC = "disc.pth.tar"
CHECKPOINT_GEN = "gen.pth.tar"

In [12]:
import torch

from torchvision.utils import save_image

def save_some_examples(gen, val_loader, epoch, folder):
    x, y = next(iter(val_loader))
    x, y = x.to(DEVICE), y.to(DEVICE)
    gen.eval()
    with torch.no_grad():
        y_fake = gen(x)
        y_fake = y_fake * 0.5 + 0.5  # remove normalization#
        save_image(y_fake, folder + f"/y_gen_{epoch}.png")
        save_image(x * 0.5 + 0.5, folder + f"/input_{epoch}.png")
        if epoch == 1:
            save_image(y * 0.5 + 0.5, folder + f"/label_{epoch}.png")
    gen.train()


def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=DEVICE)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    # and it will lead to many hours of debugging \:
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

In [13]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm


torch.backends.cudnn.benchmark = True


def train_fn(disc, gen, loader, opt_disc, opt_gen, l1_loss, bce, g_scaler, d_scaler, weighted_loss = WEIGHTED_LOSS, cm_input = INPUT_CHANGE_MAP):
    loop = tqdm(loader, leave=True)

    for idx, (s2t1,s1t1,s2t2,s1t2,cm,rcm) in enumerate(loop):
        s2t1,s1t1,s2t2,s1t2,cm,rcm = s2t1.to(DEVICE),s1t1.to(DEVICE),s2t2.to(DEVICE),s1t2.to(DEVICE),cm.to(DEVICE),rcm.to(DEVICE)
        if cm_input:
            s2t2 = torch.cat((s2t2, cm), dim=1)
            s1t1 = torch.cat((s1t1, rcm), dim=1)
        # Train Discriminator
        with torch.cuda.amp.autocast():
            s1t2_fake = gen(s2t2, s1t1)
            D_real = disc(s2t2, s1t1, s1t2)
            D_real_loss = bce(D_real, torch.ones_like(D_real))
            D_fake = disc(s2t2, s1t1, s1t2_fake.detach())
            D_fake_loss = bce(D_fake, torch.zeros_like(D_fake))
            D_loss = (D_real_loss + D_fake_loss) / 2

        disc.zero_grad()
        d_scaler.scale(D_loss).backward()
        d_scaler.step(opt_disc)
        d_scaler.update()

        # Train generator
        with torch.cuda.amp.autocast():
            D_fake = disc(s2t2, s1t1, s1t2_fake)
            G_fake_loss = bce(D_fake, torch.ones_like(D_fake))
            if weighted_loss:
                L1 = l1_loss(s1t2_fake, s1t2, cm, rcm) * L1_LAMBDA
            else:
                L1 = l1_loss(s1t2_fake, s1t2) * L1_LAMBDA
            G_loss = G_fake_loss + L1

        opt_gen.zero_grad()
        g_scaler.scale(G_loss).backward()
        g_scaler.step(opt_gen)
        g_scaler.update()

        if idx % 10 == 0:
            loop.set_postfix(
                D_real=torch.sigmoid(D_real).mean().item(),
                D_fake=torch.sigmoid(D_fake).mean().item(),
                G_loss = G_loss.item(),
                L1 = L1.item(),
            )




In [14]:
from temporalgan.temporal_gan_v3_gen import Generator as GeneratorV3
from temporalgan.temporal_gan_v2_gen import Generator as GeneratorV2
from temporalgan.temporal_gan_v1_gen import Generator as GeneratorV1
from temporalgan.temporal_gan_v2_disc import Discriminator as DiscriminatorV2
from temporalgan.temporal_gan_v1_disc import Discriminator as DiscriminatorV1
from temporalgan.lossfunciton.loss_function import WeightedL1Loss

In [15]:
def main():
    disc = DiscriminatorV1(s2_in_channels=S2_INCHANNELS, s1_in_channels=S1_INCHANNELS).to(DEVICE)
    gen = GeneratorV3(s2_in_channels=S2_INCHANNELS, s1_in_channels= S1_INCHANNELS, features=64,pam_downsample=2).to(DEVICE)
    opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999),)
    opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
    BCE = nn.BCEWithLogitsLoss()
    if WEIGHTED_LOSS:
        L1_LOSS = WeightedL1Loss(change_weight=5)
    else:
        L1_LOSS = nn.L1Loss()

    if LOAD_MODEL:
        load_checkpoint(
            CHECKPOINT_GEN, gen, opt_gen, LEARNING_RATE,
        )
        load_checkpoint(
            CHECKPOINT_DISC, disc, opt_disc, LEARNING_RATE,
        )

    transform = transforms.Compose([S2S1Normalize(),myToTensor()])


    train_dataset = Sen12Dataset(s1_t1_dir="/kaggle/input/s1s2-2021-v2/2021/s1_imgs/train",
                                s2_t1_dir="/kaggle/input/s1s2-2021-v2/2021/s2_imgs/train",
                                s1_t2_dir="/kaggle/input/s1s2-2019-v2/2019/s1_imgs/train",
                                s2_t2_dir="/kaggle/input/s1s2-2019-v2/2019/s2_imgs/train",
                                transform=transform,
                                two_way=False)
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
    )
    g_scaler = torch.cuda.amp.GradScaler()
    d_scaler = torch.cuda.amp.GradScaler()
#     val_dataset = MapDataset(root_dir=VAL_DIR)
#     val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

    for epoch in range(1, NUM_EPOCHS+1):
        print(f"Epoch: {epoch}")
        train_fn(
            disc, gen, train_loader, opt_disc, opt_gen, L1_LOSS, BCE, g_scaler, d_scaler,
        )

        if SAVE_MODEL and epoch % SAVE_EVERY_EPOCH == 0:
            save_checkpoint(epoch,gen, opt_gen, filename=CHECKPOINT_GEN)
            save_checkpoint(epoch,disc, opt_disc, filename=CHECKPOINT_DISC)

#         save_some_examples(gen, val_loader, epoch, folder="evaluation")


In [16]:
main()

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: 1


  4%|▍         | 58/1326 [00:28<10:13,  2.07it/s, D_fake=0.484, D_real=0.514, G_loss=82.9, L1=82.1]


KeyboardInterrupt: 